<a href="https://colab.research.google.com/github/AadrikaJaiswal/Apna-Guide-Tasks/blob/main/Disaster_or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords
nltk.download('stopwords')

# Load the datasets
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# Function to clean text data
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabet characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))  # Remove stopwords
    return text

# Apply text preprocessing
train_data['tweet'] = train_data['tweet'].apply(preprocess_text)
test_data['tweet'] = test_data['tweet'].apply(preprocess_text)

# Fill missing values with empty strings
train_data.fillna('', inplace=True)
test_data.fillna('', inplace=True)

# Encode categorical features
encoder = LabelEncoder()
train_data['keyword'] = encoder.fit_transform(train_data['keyword'])
test_data['keyword'] = encoder.transform(test_data['keyword'])
train_data['place'] = encoder.fit_transform(train_data['place'])

# Handle unseen labels in test data for 'place'
unseen_labels = set(test_data['place']) - set(encoder.classes_)
if unseen_labels:
    print(f"Unseen labels found in test data: {unseen_labels}")
    # Replace unseen labels with a default value
    test_data['place'] = test_data['place'].apply(lambda x: "Unknown" if x in unseen_labels else x)

# Encode 'place' feature in the test data
test_data['place'] = encoder.transform(test_data['place'])

# Vectorize the tweet text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(train_data['tweet'])
X_test_tfidf = vectorizer.transform(test_data['tweet'])

# Combine TF-IDF features with 'keyword' and 'place' features
X_train = np.hstack((X_train_tfidf.toarray(), train_data[['keyword', 'place']].values))
X_test = np.hstack((X_test_tfidf.toarray(), test_data[['keyword', 'place']].values))
y_train = train_data['disaster']

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the RandomForestClassifier model
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Validate the model
y_val_pred = classifier.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# Make predictions on the test data
test_predictions = classifier.predict(X_test)

# Create the submission file
submission = pd.DataFrame({'id': test_data['id'], 'disaster': test_predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Unseen labels found in test data: {'Redding ', 'Kentucky', 'Nigeria|| Lagos', 'started acc 1.9.15 2:25 pm', 'Coos Bay, OR', 'Nigeria,West African', 'patna', 'Somewhere in the Internet...', 'Time for NON VIOLENT dissent! ', 'SW London (RBK)', 'Paranaque City, National Capital Region', 'Hollywood, Ca', 'sam', 'Ohio land of no sun', 'Rochester NY', "Ton's Ì¡ead town åÈ Tx", 'L.A', 'US of Eh', 'Indian Trail, North Carolina', '???/??', 'Currently Somewhere On Earth', 'Elko, Nevada', 'Minneapolis St. Paul Minnesota', '$$$', 'Northern Scandinavia', '408 N. Western, Wenatchee, WA ', 'Dallas, Texas', 'Waterloo, Ontario, Canada', 'uchiha', 'Oregon & SW Washington', 'Barnsley', 'Panama City Beach, Florida', 'trashcan somewhere in hell', 'Hustle Flow Nation', 'trust none..', '?? | pittsburgh ', 'Unauthentically British UK', 'The Woodlands, TX', 'Hodesto, Cuntlifornia', 'Chattanooga, TN', 'Albequerque', 'Eugene, OR.', 'Kocaeli-Izmit', "Maryland/Myrtle Beach CCU'19", 'Paradise ', 'Southend-on-Sea, E